In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ['GOOGLE_API_KEY'] = os.getenv('GOOGLE_API_KEY')
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')
os.environ['LANGCHAIN_TRACING_V2'] = "true"
os.environ['LANGCHAIN_PROJECT'] = os.getenv('LANGCHAIN_PROJECT')



In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)




/home/codespace/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
#Input and get response from LLM
result = llm.invoke("What is MDR in credit card?")
print(result)

content="MDR in credit card stands for **Merchant Discount Rate**. It's the fee a merchant (business) pays to their bank or payment processor for the service of accepting credit and debit card payments from customers.\n\nHere's a breakdown:\n\n*   **Who pays it?** The merchant (the business selling the goods or services).\n*   **Who receives it?** It's typically split between the acquiring bank (the merchant's bank), the card network (Visa, Mastercard, etc.), and the issuing bank (the customer's bank).\n*   **What does it cover?** It covers the costs associated with processing the transaction, including:\n    *   **Interchange fees:** These are set by the card networks and paid to the issuing bank to cover the risk of fraud and other costs. Interchange fees make up the largest portion of the MDR.\n    *   **Assessment fees:** These are fees charged by the card networks (Visa, Mastercard, etc.) for using their brand and network.\n    *   **Processor fees:** These are fees charged by the

In [6]:
## Prompt Template - What kind of roleplaying you want the LLM to do.
from langchain_core.prompts import ChatPromptTemplate

In [8]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system","""You an expert in text cleaning. You also have sound knowledge of all Hong Kong merchants.
         You have experience in recognizing merchant names from uncleaned or shortened merchant names and is able to seperate a retail merchant from payment merchants.
         Provide me your best answer that you can."""),
         ("user","""{input}""")

    ]
)

In [15]:
# chaining

chain = prompt|llm
results = chain.invoke({"input":"Clean the following merchant names and resturn in table format with key like Sr.No|Retail_Mrch|Payment_Mrch, ['CATHPAC002','TAOBAO','CHANELIPP12M','ALIPAY*UBER','PAYPAL*UBER','APPLIPP12','MCD*GPAY'] "})

In [16]:
print(results.content)

Okay, I can help you clean those merchant names and identify retail versus payment merchants. Here's the table with the cleaned and categorized merchant names:

| Sr.No | Retail_Mrch | Payment_Mrch |
|---|---|---|
| 1 | Cathay Pacific |  |
| 2 | Taobao |  |
| 3 | Chanel |  |
| 4 | Uber | Alipay |
| 5 | Uber | PayPal |
| 6 | Apple |  |
| 7 | McDonald's | Google Pay |

**Explanation of Cleaning and Categorization:**

*   **CATHPAC002 -> Cathay Pacific:**  "CATHPAC" is a common abbreviation for Cathay Pacific Airways, Hong Kong's major airline.
*   **TAOBAO -> Taobao:** Taobao is a large online shopping platform, similar to eBay or Amazon.
*   **CHANELIPP12M -> Chanel:** "CHANEL" is a luxury fashion brand.
*   **ALIPAY\*UBER -> Uber | Alipay:**  "ALIPAY\*" indicates the transaction was processed through Alipay, a digital wallet/payment platform. "UBER" indicates the retail merchant.
*   **PAYPAL\*UBER -> Uber | PayPal:** "PAYPAL\*" means the transaction went through PayPal, another digita

In [20]:
#Output parser
from langchain_core.output_parsers import StrOutputParser

In [21]:
out_parser = StrOutputParser()

In [22]:
chain = prompt|llm|out_parser
results = chain.invoke({"input":"Clean the following merchant names and resturn in table format with key like Sr.No|Retail_Mrch|Payment_Mrch, ['CATHPAC002','TAOBAO','CHANELIPP12M','ALIPAY*UBER','PAYPAL*UBER','APPLIPP12','MCD*GPAY'] "})

In [24]:
print(results)

Okay, I can help you clean those merchant names and identify retail versus payment merchants. Here's the table with the cleaned and categorized merchant names:

| Sr.No | Retail_Mrch | Payment_Mrch |
|---|---|---|
| 1 | Cathay Pacific |  |
| 2 | Taobao |  |
| 3 | Chanel |  |
| 4 | Uber | Alipay |
| 5 | Uber | PayPal |
| 6 | Apple |  |
| 7 | McDonald's | Google Pay |

**Explanation of Cleaning and Categorization:**

*   **CATHPAC002 -> Cathay Pacific:**  "CATHPAC" is a common abbreviation for Cathay Pacific Airways, Hong Kong's major airline.
*   **TAOBAO -> Taobao:** Taobao is a large online shopping platform, similar to eBay or Amazon.
*   **CHANELIPP12M -> Chanel:** "CHANEL" is a luxury fashion brand.
*   **ALIPAY\*UBER -> Uber | Alipay:**  "ALIPAY\*" indicates the transaction was processed through Alipay, a digital wallet/payment platform. "UBER" indicates the retail merchant.
*   **PAYPAL\*UBER -> Uber | PayPal:** "PAYPAL\*" means the transaction went through PayPal, another digita